In [3]:
import findspark
findspark.init("c:\spark")

In [4]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext

In [5]:
conf = SparkConf() \
.setMaster("local[4]") \
.setAppName("Test") \
.setExecutorEnv("spark.executor.memory" , "3g") \
.setExecutorEnv("spark.driver.memory" , "2g")

In [6]:
sc = SparkContext(conf=conf)

In [8]:
dataRDD = sc.textFile("C:\\Users\\MERT\\Untitled Folder\\Data\\OnlineRetail.csv")

In [9]:
dataRDD.take(3)

['InvoiceNo;StockCode;Description;Quantity;InvoiceDate;UnitPrice;CustomerID;Country',
 '536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom']

In [10]:
## Su başlıkkları temizliyelim

firstLine = dataRDD.first()
firstLineRdd = sc.parallelize([firstLine])


In [11]:
noHeader_RDD = dataRDD.subtract(firstLineRdd)

In [13]:
noHeader_RDD.take(5)

['536367;84969;BOX OF 6 ASSORTED COLOUR TEASPOONS;6;1.12.2010 08:34;4,25;13047;United Kingdom',
 '536369;21756;BATH BUILDING BLOCK WORD;3;1.12.2010 08:35;5,95;13047;United Kingdom',
 '536370;22326;ROUND SNACK BOXES SET OF4 WOODLAND;24;1.12.2010 08:45;2,95;12583;France',
 '536370;21731;RED TOADSTOOL LED NIGHT LIGHT;24;1.12.2010 08:45;1,65;12583;France',
 '536372;22632;HAND WARMER RED POLKA DOT;6;1.12.2010 09:01;1,85;17850;United Kingdom']

## Birim miktari 30'dan büyük olanları filtreleme

In [16]:
noHeader_RDD.filter(lambda x : int(x.split(";")[3]) > 30).take(5)

['536378;85183B;CHARLIE & LOLA WASTEPAPER BIN FLORA;48;1.12.2010 09:37;1,25;14688;United Kingdom',
 '536381;22719;GUMBALL MONOCHROME COAT RACK;36;1.12.2010 09:41;1,06;15311;United Kingdom',
 '536384;22470;HEART OF WICKER LARGE;40;1.12.2010 09:53;2,55;18074;United Kingdom',
 '536386;85099C;JUMBO  BAG BAROQUE BLACK WHITE;100;1.12.2010 09:57;1,65;16029;United Kingdom',
 '536387;79321;CHILLI LIGHTS;192;1.12.2010 09:58;3,82;16029;United Kingdom']

## Cofee isimli olanları filtreleme

In [18]:
noHeader_RDD.filter(lambda x : "COFFEE" in x.split(";")[2]).take(9)

['536739;85159A;BLACK TEA,COFFEE,SUGAR JARS;2;2.12.2010 13:08;6,35;14180;United Kingdom',
 '536750;37370;RETRO COFFEE MUGS ASSORTED;6;2.12.2010 14:04;1,06;17850;United Kingdom',
 '536787;37370;RETRO COFFEE MUGS ASSORTED;6;2.12.2010 15:24;1,06;17850;United Kingdom',
 '536804;37370;RETRO COFFEE MUGS ASSORTED;72;2.12.2010 16:34;1,06;14031;United Kingdom',
 '536805;37370;RETRO COFFEE MUGS ASSORTED;12;2.12.2010 16:38;1,25;14775;United Kingdom',
 '536864;21216;SET 3 RETROSPOT TEA,COFFEE,SUGAR;1;3.12.2010 11:27;11,02;000000;United Kingdom',
 '536865;37370;RETRO COFFEE MUGS ASSORTED;1;3.12.2010 11:28;16,13;000000;United Kingdom',
 '537126;21216;SET 3 RETROSPOT TEA,COFFEE,SUGAR;1;5.12.2010 12:13;4,95;18118;United Kingdom',
 '537231;22304;COFFEE MUG BLUE PAISLEY DESIGN;6;6.12.2010 09:21;2,55;13652;United Kingdom']

In [19]:
def pair_filter(x):
    quantity = int(x.split(";")[3])
    description = x.split(";")[2]
    
    return (quantity >30) & ("COFFEE" in description)

In [22]:
noHeader_RDD.filter(lambda x: pair_filter(x)).take(10)

['536804;37370;RETRO COFFEE MUGS ASSORTED;72;2.12.2010 16:34;1,06;14031;United Kingdom',
 '540176;22302;COFFEE MUG PEARS  DESIGN;36;5.01.2011 12:40;2,1;17511;United Kingdom',
 '540459;22970;LONDON BUS COFFEE MUG;36;7.01.2011 12:35;2,1;16191;United Kingdom',
 '541597;37370;RETRO COFFEE MUGS ASSORTED;216;19.01.2011 16:19;1,06;14031;United Kingdom',
 '542373;22303;COFFEE MUG APPLES DESIGN;36;27.01.2011 13:38;2,1;16767;United Kingdom',
 '542789;22970;LONDON BUS COFFEE MUG;36;1.02.2011 10:38;2,1;17511;United Kingdom',
 '543054;22300;COFFEE MUG DOG + BALL DESIGN;36;3.02.2011 10:39;2,1;13394;United Kingdom',
 '543594;22970;LONDON BUS COFFEE MUG;36;10.02.2011 12:15;2,1;18093;United Kingdom',
 '544164;22971;QUEENS GUARD COFFEE MUG;36;16.02.2011 13:02;2,1;17306;United Kingdom',
 '544480;22303;COFFEE MUG APPLES DESIGN;72;21.02.2011 10:32;2,1;14646;Netherlands']